In [4]:
import polars as pl
import polars.selectors as cs
import sklearn
import catboost

import warnings 
warnings.filterwarnings('ignore')

In [5]:
print('1. Data Exploration')

1. Data Exploration


In [6]:
# King County House Sales dataset from OpenML (includes Seattle)
# this is an ARFF file, which is a text file with a specific format
url = 'https://www.openml.org/data/download/22044765/dataset'
cols = ['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 
        'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
        'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'date_year', 'date_month', 'date_day']

raw = pl.read_csv(url, new_columns=cols, skip_rows=31, has_header=False) # some rows are skipped due to dataset format

# data summary
#display(raw)
raw.describe() 

statistic,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_year,date_month,date_day
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0,21613.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",4.5803e9,540088.141767,3.370842,2.114757,2079.899736,15106.967566,1.494309,0.007542,0.234303,3.40943,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652,2014.322954,6.574423,15.688197
"""std""",2.8766e9,367127.196483,0.930062,0.770163,918.440897,41420.511515,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.67924,53.505026,0.138564,0.140828,685.391304,27304.179631,0.467616,3.115308,8.635063
"""min""",1.000102e6,75000.0,0.0,0.0,290.0,520.0,1.0,0.0,0.0,1.0,1.0,290.0,0.0,1900.0,0.0,98001.0,47.1559,-122.519,399.0,651.0,2014.0,1.0,1.0
"""25%""",2.1230e9,321950.0,3.0,1.75,1427.0,5040.0,1.0,0.0,0.0,3.0,7.0,1190.0,0.0,1951.0,0.0,98033.0,47.471,-122.328,1490.0,5100.0,2014.0,4.0,8.0
"""50%""",3.9049e9,450000.0,3.0,2.25,1910.0,7618.0,1.5,0.0,0.0,3.0,7.0,1560.0,0.0,1975.0,0.0,98065.0,47.5718,-122.23,1840.0,7620.0,2014.0,6.0,16.0
"""75%""",7.3089e9,645000.0,4.0,2.5,2550.0,10688.0,2.0,0.0,0.0,4.0,8.0,2210.0,560.0,1997.0,0.0,98118.0,47.678,-122.125,2360.0,10083.0,2015.0,9.0,23.0
"""max""",9.9000e9,7.7e6,33.0,8.0,13540.0,1.651359e6,3.5,1.0,4.0,5.0,13.0,9410.0,4820.0,2015.0,2015.0,98199.0,47.7776,-121.315,6210.0,871200.0,2015.0,12.0,31.0


In [7]:
# Data visualization

print("1) correlation")
display(raw
        .to_pandas(use_pyarrow_extension_array=True)
        .corr()
        .style.background_gradient(cmap='RdBu', vmin=-1, vmax=1)
       )

print("2) scatter plot")
display(raw
        .plot.scatter('sqft_living', 'price', alpha=0.1)
       )

print("3) plot the trend of price by date, grouped in zip code")
display(raw
        .group_by('date_month', 'zipcode')
        .agg(pl.col('price').mean())
        .sort('date_month') #sort by date
        .plot.line('date_month', 'price', by='zipcode', alpha=0.5)
       )

print("4) lat/long scatter plot")
# observed that prices are higher around the coast
display(raw
        #.filter(pl.col('price') > 1_000_000) # to display luxurious houses better, filter out house prices below $1 Mil. 
        .sort('price')
        .plot.scatter(x='long', y='lat', alpha=0.5, c='price', s=1) # this display prices on a map
       )

1) correlation


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_year,date_month,date_day
id,1.000000,-0.016762,0.001286,0.005160,-0.012258,-0.132109,0.018525,-0.002721,0.011592,-0.023783,0.008130,-0.010842,-0.005151,0.021380,-0.016907,-0.008224,-0.001891,0.020799,-0.002901,-0.138798,0.010128,-0.011708,0.001939
price,-0.016762,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447,0.003576,-0.010081,-0.014670
bedrooms,0.001286,0.308350,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244,-0.009838,-0.001533,-0.008031
bathrooms,0.005160,0.525138,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175,-0.026596,0.007392,-0.005304
sqft_living,-0.012258,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286,-0.029038,0.011810,-0.007246
sqft_lot,-0.132109,0.089661,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557,0.005468,-0.002369,0.000634
floors,0.018525,0.256794,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269,-0.022315,0.014005,-0.006662
waterfront,-0.002721,0.266369,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703,-0.004165,0.008132,0.010518
view,0.011592,0.397293,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575,0.001364,-0.005638,0.011174
condition,-0.023783,0.036362,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406,-0.045589,0.021978,-0.004643


2) scatter plot


:Scatter   [sqft_living]   (price)

3) plot the trend of price by date, grouped in zip code


:NdOverlay   [zipcode]
   :Curve   [date_month]   (price)

4) lat/long scatter plot


:Scatter   [long]   (lat,price)

In [8]:
print('2. Data Preprocessing')

2. Data Preprocessing


In [9]:
# import sklearn pipelines

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer #ColumnTransformer is a tool that allows us to run certain steps just on specific columns, not on everything in there
from sklearn.preprocessing import StandardScaler, OneHotEncoder #StandardScaler standardizes the data, meaning it gives each column a mean value of zero and a standard deviation of one; OneHotEncoder is a mechanism for taking categorical data, because most machine learning algorithms don't work with text data or categorical data, and it encodes that into numeric values
from sklearn.impute import SimpleImputer #SimpleImputer that fills in missing values
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer #FunctionTransformer is a class from Scikit-Learn that allows converting a function into a transformer to stick into a pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import set_config
set_config(transform_output='polars')

In [10]:
# 1.1 handle numerical variables
#print(tweak_housing(raw).select(cs.numeric()).columns) # identify numerical columns in the dataset
numeric_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 
                    'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 
                    'lat', 'long', 'sqft_living15', 'sqft_lot15', 'zip_mean', 'price']  # note that zip_mean will be added in step 3.0 as the average price of a zipcode
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())]) # define numerical transformer

# 1.2 handle categorical variables
categorical_features = ['zipcode']
categorical_transformer = OneHotEncoder(handle_unknown='ignore',
                                        sparse_output=False, max_categories=10) # allow only 10 categorical vars to be created instead of high-dimentional zip codes


# *1.3 Column Transformer: apply numerical transformation to number columns and categorical transformer to categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [11]:
# 2.1 Tweak function: define a function to tweak housing data by condensing date information and adjusting renovated years
def tweak_housing(df):
    return (df
            .with_columns(zipcode=pl.col('zipcode').cast(pl.String).cast(pl.Categorical),
                          date=pl.date(pl.col('date_year'), pl.col('date_month'), pl.col('date_day')),
                          yr_renovated=pl.col('yr_renovated').replace(0, None),
                          )
            .select(['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 
                     'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 
                     'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 
                     'sqft_lot15', 'date',  #'date_year', 'date_month', 'date_day', 
                     ])
    )
#tweak_housing(raw)

# *2.2 Treak Transformer: Treak function as a transformer
tweak_transformer = FunctionTransformer(tweak_housing)

In [12]:
# *3.0 Custom Transformer
class ZipAvgPriceAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        # assume X is a polars dataframe
        self.zip_avg_price = (X
                              .group_by('zipcode')
                              .agg(zip_mean=pl.col('price').mean()) # get the average price of each zip code
        )
        return self
    
    def transform(self, X, y=None):
        return X.join(self.zip_avg_price, on='zipcode') # add zip average price to columns of X

#zip_adder = ZipAvgPriceAdder()
#zip_adder.fit_transform(raw.select(['zipcode', 'price'])) 


In [13]:
# 4.0 Make the Pipeline!
# Append classifier to preprocessing pipeline. Now we have a full prediction pipeline.
pipe = Pipeline(steps=[('tweak', tweak_transformer),
                      ('zip_avg_price', ZipAvgPriceAdder()),
                      ('preprocessor', preprocessor),
                      ])

X = raw #.drop('price')
y = raw.select('price') # Note sklearn wants a Polars dataframe for y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# outputs:
display(pipe)
df_transformed = pipe.fit_transform(raw)
#df_transformed = pipe.fit_transform(raw, raw.select('price'))
display(df_transformed) # Standardize features



Pipeline(steps=[('tweak',
                 FunctionTransformer(func=<function tweak_housing at 0x0000018107AC2D40>)),
                ('zip_avg_price', ZipAvgPriceAdder()),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['bedrooms', 'bathrooms',
                                                   'sqft_living', 'sqft_lot',
                                                   'floors', 'waterfront',
                                                   'view', 'condition', 'grade',
                                                   'sqft_above',
                                                   'sqft_basement', 'yr_built',
                                                   'yr_renovated', 'lat',
                                                   'long', 'sqft_living15',
                                                   'sqft_lot15', 'zip_mean',
                                                   'price']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                max_categories=10,
                                                                sparse_output=False),
                                                  ['zipcode'])]))])

num__bedrooms,num__bathrooms,num__sqft_living,num__sqft_lot,num__floors,num__waterfront,num__view,num__condition,num__grade,num__sqft_above,num__sqft_basement,num__yr_built,num__yr_renovated,num__lat,num__long,num__sqft_living15,num__sqft_lot15,num__zip_mean,num__price,cat__zipcode_98023,cat__zipcode_98034,cat__zipcode_98038,cat__zipcode_98042,cat__zipcode_98052,cat__zipcode_98103,cat__zipcode_98115,cat__zipcode_98117,cat__zipcode_98118,cat__zipcode_infrequent_sklearn
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-0.398737,-1.447464,-0.979835,-0.228321,-0.915427,-0.087173,-0.305759,-0.629187,-0.558836,-0.734708,-0.658681,-0.544898,0.053508,-0.352572,-0.306079,-0.943355,-0.260715,-0.979321,-0.866717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
-0.398737,0.175607,0.533634,-0.189885,0.936506,-0.087173,-0.305759,-0.629187,-0.558836,0.460841,0.245141,-0.681079,-2.67545,1.161568,-0.746341,-0.432686,-0.187868,-0.301434,-0.005688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
-1.473959,-1.447464,-1.426254,-0.123298,-0.915427,-0.087173,-0.305759,-0.629187,-1.409587,-1.229834,-0.658681,-1.293892,0.053508,1.283537,-0.135655,1.07014,-0.172375,-0.331204,-0.980849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0.676485,1.149449,-0.13055,-0.244014,-0.915427,-0.087173,-0.305759,2.444294,-0.558836,-0.891699,1.397515,-0.204446,0.053508,-0.283288,-1.271816,-0.914174,-0.284522,0.049507,0.17409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
-0.398737,-0.149007,-0.435422,-0.169653,-0.915427,-0.087173,-0.305759,-0.629187,0.291916,-0.130895,-0.658681,0.544548,0.053508,0.40955,1.199335,-0.27219,-0.192849,0.621019,-0.081958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
-0.398737,0.500221,-0.598746,-0.337424,2.788439,-0.087173,-0.305759,-0.629187,0.291916,-0.312039,-0.658681,1.293542,0.053508,1.004958,-0.938069,-0.666135,-0.412381,0.191323,-0.490545,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
0.676485,0.500221,0.250539,-0.224386,0.936506,-0.087173,-0.305759,-0.629187,0.291916,0.629908,-0.658681,1.463768,0.053508,-0.35618,-1.051685,-0.228419,-0.203946,-0.770759,-0.381588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
-1.473959,-1.772078,-1.154047,-0.332137,0.936506,-0.087173,-0.305759,-0.629187,-0.558836,-0.927928,-0.658681,1.293542,0.053508,0.247888,-0.604321,-1.410253,-0.394141,0.232414,-0.375865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
print("Additional Analysis")

Additional Analysis


In [16]:
display(df_transformed
        .plot.scatter('num__bedrooms', 'num__zip_mean', alpha=0.01) 
        # observed that num__zip_mean is a discrete varaible, because it is grouped as average price by zip lable
       )
display(df_transformed
        .plot.scatter('num__bedrooms', 'num__price', alpha=0.01) # num__price is continuous
       )


:Scatter   [num__bedrooms]   (num__zip_mean)

:Scatter   [num__bedrooms]   (num__price)

In [28]:
display(raw
        .plot.scatter('bedrooms', 'price', alpha=0.1, yformatter='%.0f')
        .opts(jitter=.8)
       )

:Scatter   [bedrooms]   (price)